In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!unzip -q '/content/drive/MyDrive/Breast_Cancer.zip' -d '/content/Breast_Cancer'

replace /content/Breast_Cancer/Breast Cancer/test/breast_benign/breast_benign_0009.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [17]:
import os

base_dir = '/content/Breast_Cancer'
for root, dirs, files in os.walk(base_dir):
  print(root, 'contains', len(files), 'files')

/content/Breast_Cancer contains 0 files
/content/Breast_Cancer/Breast Cancer contains 0 files
/content/Breast_Cancer/Breast Cancer/train contains 0 files
/content/Breast_Cancer/Breast Cancer/train/breast_malignant contains 4000 files
/content/Breast_Cancer/Breast Cancer/train/breast_benign contains 4000 files
/content/Breast_Cancer/Breast Cancer/test contains 0 files
/content/Breast_Cancer/Breast Cancer/test/breast_malignant contains 1000 files
/content/Breast_Cancer/Breast Cancer/test/breast_benign contains 1000 files


In [18]:
import matplotlib.pyplot as plt
import cv2
import random

def show_random_images(folder, n=5):
  imgs = os.listdir(folder)
  plt.figure(figsize=(15, 3))
  for i in range(n):
    img_name = random.choice(imgs)
    img_path = os.path.join(folder, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(1, n, i+1)
    plt.imshow(img)
    plt.axis('off')
  plt.show()

  show_random_images('/content/Breast_Cancer/train/benign')
  show_random_images('/content/Breast_Cancer/train/malignant')

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [20]:
train_dir = '/content/Breast_Cancer/Breast Cancer/train'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(512, 512),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(512, 512),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 6400 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.


In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [22]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='sigmoid'),
    layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 510, 510, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 255, 255, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 253, 253, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 126, 126, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 124, 124, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 492032)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │   125,960,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 126,053,953 (480.86 MB)

 Trainable params: 126,053,953 (480.86 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=2
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
200/200 ━━━━━━━━━━━━━━━━━━━━ 4677s 23s/step - accuracy: 0.5011 - loss: 1.8225 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/2
 41/200 ━━━━━━━━━━━━━━━━━━━━ 57:19 22s/step - accuracy: 0.4766 - loss: 0.6940

In [ ]:
plt.figure(figsize=(14,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.legend('Train', 'Validation')

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.legend('Train', 'Validation')

plt.show()

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
img_path = '/content/Breast_Cancer/Breast Cancer/test/benign/benign (1).png'
img = cv2.imread(img_path)
if img is None:
  print(f'Error: Could not load image from {img_path}')
else:
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.imshow(img)
  plt.axis('off')
  img_resized = cv2.resize(img, (512, 512)) / 255.0
  img_tensor = np.expand_dims(img_resized, axis=0)
  prediction = model.predict(img_tensor)[0][0]
  if prediction > 0.5:
    print('Prediction: Malignant')
  else:
    print('Prediction: Benign')

In [ ]:
model.save('/content/drive/MyDrive/Breast_Cancer_Model.h5')

In [ ]:
!cp '/content/Breast_Cancer_Model.h5' '/content/drive/MyDrive/Breast_Cancer_Model.h5'

In [ ]:
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()